In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz
from sklearn.model_selection import GridSearchCV
from graphviz import Source
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from IPython.display import display



# 1. Understand the Problem and Data

Key questions to answer:
- What is the current churn rate?
- Are there patterns or trends in employee departures?
- What are the factors influencing churn (e.g., department, tenure, performance)?
- Can we predict churn and design retention strategies?

In [ ]:
df = pd.read_csv("employee_retention.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().mean().mul(100).to_dict()

In [ ]:
for col in df.columns:
    uniques = df[col].unique()
    print(f"{col:<15}{len(uniques):<10}{uniques[:5]}")

# 2. Data Preparation

In [ ]:
df["join_date"] = pd.to_datetime(df["join_date"])
df["quit_date"] = pd.to_datetime(df["quit_date"])

In [ ]:
df.describe(include='all')

In [ ]:
df = df[~(df["seniority"]> 60)]

In [ ]:
df['tenure'] = (df['quit_date'].fillna(pd.Timestamp(datetime.now())) - df["join_date"]).dt.days
df["churn"] = df["quit_date"].apply(lambda x: 0 if pd.isna(x) else 1) # 1 = churned, 0 = active

In [ ]:
df.sample(3)

In [ ]:
join_df = df[["company_id", "join_date"]].rename(columns={"join_date" : "day"})
quit_df = df[["company_id", "quit_date"]].rename(columns={"quit_date" : "day"}).dropna()

In [ ]:
join_df["change"] = 1
quit_df["change"] = -1

In [ ]:
union_df = pd.concat([join_df, quit_df])

In [ ]:
company_headcount_df = union_df.groupby(["company_id","day"]).sum().reset_index()
company_headcount_df["employee_headcount"] = company_headcount_df.groupby("company_id")["change"].cumsum()
company_headcount_df["day"] = pd.to_datetime(company_headcount_df["day"])

In [ ]:
# palette = sns.color_palette("husl", n_colors=len(company_headcount_df["company_id"].unique()))

plt.figure(figsize=(20, 4))
sns.lineplot(
    data=company_headcount_df,
    x="day",
    y="employee_headcount",
    hue="company_id",
    palette="tab20"
)
plt.title("Employee Headcount Trends by Company", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Headcount", fontsize=12)
plt.legend(title="Company ID")
plt.grid(linestyle="--", alpha=0.6)
plt.show()

In [ ]:
g = sns.FacetGrid(
    company_headcount_df,
    col="company_id",
    col_wrap=4,
    height=4,
    sharey=True,
    hue="company_id",
)
g.map_dataframe(sns.lineplot, x="day", y="employee_headcount", color="blue")
# g.set_titles("Company {col_name}")
g.set_axis_labels("Date", "Headcount")
g.fig.suptitle("Employee Headcount Trends by Company", fontsize=16)
plt.subplots_adjust(top=0.9)
plt.show()


In [ ]:
join_data = join_df.groupby(["company_id","day"]).sum().reset_index().rename(columns={'change':'inflow'})
quit_data = quit_df.groupby(["company_id","day"]).sum().abs().reset_index().rename(columns={'change':'outflow'})

In [ ]:
change_df = pd.merge(join_data,quit_data, on=['day', 'company_id'] , how="outer").fillna(0)

In [ ]:
data = change_df.groupby(["company_id", change_df["day"].dt.to_period("M")])[["inflow", "outflow"]].sum().reset_index()

In [ ]:
data["day"] = data["day"].dt.to_timestamp()

In [ ]:
data.info()

In [ ]:
# Define the plotting function
def plot_with_fill(data, **kwargs):
    ax = plt.gca()
    x = data["day"].to_numpy()
    inflow = data["inflow"].to_numpy()
    outflow = data["outflow"].to_numpy()

    ax.plot(x, inflow, label="Inflow", color="blue")
    ax.plot(x, outflow, label="Outflow", color="orange")
    ax.fill_between(x, inflow, outflow, color="gray", alpha=0.2)

# Apply FacetGrid
g = sns.FacetGrid(data=data, col="company_id", col_wrap=4, height=4)
g.map_dataframe(plot_with_fill)

# Format x-axis
for ax in g.axes.flat:
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
    ax.tick_params(axis="x")

g.add_legend()
plt.show()

# 3. Exploratory Data Analysis

Use visualizations to explore trends and patterns:
- Churn over time: Plot monthly or yearly churn rates.
- Segment analysis: Compare churn rates across departments, seniority levels, or performance scores.
    - Department
    - Tenure groups (e.g., 0-1 year, 1-3 years)
    - Salary bands
    - Seniority
- Correlation analysis: Examine relationships between variables (e.g., low salary and high churn).

## a. Univariate Analysis

###  quit_date

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
sns.histplot(data=df[df["churn"]==1], x="tenure", bins=100, ax=ax, color='skyblue')
ax.set_title('Histogram of Tenure', fontsize=16)
ax.set_xlabel('Tenure', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
plt.show()

Most employees leave within 0–400 days, indicating early churn is a significant issue.

In [ ]:
# Week No. for quiting
quit_week = quit_data['day'].dt.isocalendar().week.astype(int)

fig, ax = plt.subplots(figsize=(20, 5))
sns.countplot(quit_week, ax=ax)
ax.set_title('Countplot of Quit Week No.')
ax.set_xlabel('Week Number')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
# conditions = [
#     (df["tenure"] <= 200),
#     (df["tenure"] >= 201) & (df["tenure"] <= 400),
#     (df["tenure"] >= 401) & (df["tenure"] <= 800),
#     (df["tenure"] >= 801) & (df["tenure"] <= 1200),
#     (df["tenure"] >= 1201),
# ]

# choices = ["0-200 days", "201-400 days", "401-800 days", "801-1200 days", "1201+ days"] 

# df['tenure_grp'] = np.select(conditions, choices, default='Unknown')


### salary

In [ ]:
# KDE plot: Compare salary distributions
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
sns.kdeplot(data=df, x="salary", hue="churn", fill=True, common_norm=False, ax=ax[0])
ax[0].set_title("Salary Distribution by Churn Status")
ax[0].set_xlabel("Salary")
ax[0].set_ylabel("Density")

# Boxplot: Compare salary distributions
sns.boxplot(data=df, x="churn", y="salary", ax=ax[1])
ax[1].set_title("Boxplot of salary by Churn Status")
ax[1].set_xlabel("Churn Status")
ax[1].set_ylabel("Salary")

plt.tight_layout()
plt.show()


### seniority

In [ ]:
# KDE plot: Compare seniority distributions
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
sns.kdeplot(data=df, x="seniority", hue="churn", fill=True, common_norm=False, ax=ax[0])
ax[0].set_title("seniority Distribution by Churn Status")
ax[0].set_xlabel("seniority")
ax[0].set_ylabel("Density")

# Boxplot: Compare seniority distributions
sns.boxplot(data=df, x="churn", y="seniority", ax=ax[1])
ax[1].set_title("Boxplot of seniority by Churn Status")
ax[1].set_xlabel("Churn Status")
ax[1].set_ylabel("seniority")

plt.tight_layout()
plt.show()


### dept

In [ ]:
df.dept.value_counts()

In [ ]:
col_to_plot = 'dept'
fig, ax = plt.subplots(1,2,figsize=(20,5))

sns.countplot(data=df, x=col_to_plot, hue='churn', ax=ax[0])
ax[0].set_title(f"Count Plot of {col_to_plot}")
sns.barplot(data=df, x=col_to_plot, y='churn', ax=ax[1])
ax[1].set_title(f'Churn Rate per {col_to_plot}')
ax[1].set_ylabel("Churn Rate")
plt.tight_layout()
plt.show()

- aspect=1 (default): Each subplot's width equals its height, providing a balanced layout.
- aspect=2: Subplots are twice as wide as they are tall, useful for plots with longer labels or more detail.
- aspect=0.5: Subplots are narrower, which can help fit more plots horizontally in a limited space.

In [ ]:
g = sns.FacetGrid(data=df, col='company_id', col_wrap=4, height=6, aspect=1.1)
g.map_dataframe(sns.barplot, x="dept", y="churn")

plt.show()

In [ ]:
df.columns

In [ ]:
df.head()

# 4. Feature Engineering

- Tenure in months.
- Salary percentile.
- Number of training sessions attended.
- Performance score trends.
- Department-level churn rate.

In [ ]:
# One-hot encode categorical columns
df = pd.get_dummies(df, columns=['dept'], drop_first=True)

In [ ]:
df.sample(3)

# 5. Modeling

- Logistic Regression: Predict probability of churn.
- Decision Trees/Random Forests: Identify key churn drivers.
    - Analyze feature importance (from models) or significant correlations:
        - If low salary and low performance scores are strong churn predictors, these could be key focus areas for retention.
- XGBoost/LightGBM: For high-performance churn prediction.
- Clustering (e.g., k-means): Segment employees to identify high-risk groups.

In [ ]:
# Drop unnecessary columns
data = df.drop(columns=['employee_id', 'join_date', 'quit_date'])

In [ ]:
data.churn.value_counts()

In [ ]:
# Split the data into features and target
X = data.drop(columns=['churn','tenure'])
y = data['churn']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Decision Tree

In [ ]:
# Initialize the Decision Tree Classifier
model = DecisionTreeClassifier(
    random_state=42,
    max_depth=3,
    min_samples_leaf=30,
)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Print metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


- Precision: The ability to avoid false positives. (High precision = fewer false positives.)
- Recall: The ability to identify true positives. (High recall = fewer false negatives.)
- F1-score: The harmonic mean of precision and recall. It balances false positives and false negatives.

- Look at the classification report:
    - Churn class (1): Recall is high (0.84), but precision is lower (0.58).
    - Non-churn class (0): Precision is low (0.53), and recall is even lower (0.24).
    - Imbalance in recall and precision suggests the model performs better for predicting churn but struggles with non-churn cases.
- Accuracy:
    - An accuracy of 57% suggests the model's overall performance is slightly better than random guessing (which would be 50% for binary classification).

- If the cost of misclassification is higher for one class (e.g., predicting a churned user as not churned), prioritize recall for that class.
- The decision tree can help stakeholders understand the factors driving churn (e.g., salary and seniority) and guide actionable interventions.

In [ ]:
dot_data = export_graphviz(
    model, 
    out_file=None,  # Specifies the output file where the .dot (Graphviz) representation of the tree should be written. Setting this to None means the function will return the graph as a string instead of saving it to a file.
    feature_names=X.columns, 
    class_names=["Not Churned", "Churned"], # "Not Churned" is for class 0, and "Churned" is for class 1. The intensity of the color indicates the proportion of samples in the node belonging to the predicted class.
    filled=True, # Colors the nodes based on the class they represent and the majority class in that node.
    # rounded=True, # Draws the nodes with rounded corners for better visual aesthetics.
    special_characters=True # Allows special characters (like ≥ or ≤) to be used in the text labels for nodes.
)

graph = Source(dot_data)
display(graph)

In [ ]:
# plt.figure(figsize=(20, 10))
# plot_tree(model, feature_names=X.columns, class_names=["Not Churned", "Churned"], filled=True)
# plt.show()


In [ ]:
importances = model.feature_importances_
features = X.columns

plt.barh(features, importances)
plt.xlabel("Importance")
plt.ylabel("Features")
plt.show()

In [ ]:
param_grid = {
    "max_depth": [3, 5, 10, 20, None],
    "min_samples_split": [2, 5, 10, 20], # Larger values ensure splits occur only with enough data, reducing overfitting.
    "min_samples_leaf": [1, 2, 5, 10], # Larger values ensure splits occur only with enough data, reducing overfitting.
    "criterion": ["gini", "entropy"], # "gini" is faster, while "entropy" provides more information-theoretic splits.
    "class_weight": ['balanced', None]
}


# Initialize the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_model,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring="accuracy",  # Use accuracy as the evaluation metric
    verbose=2,
    n_jobs=-1  # Use all available processors
)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Evaluate the best model on the test data
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
dot_data = export_graphviz(
    best_model, 
    out_file=None,  # Specifies the output file where the .dot (Graphviz) representation of the tree should be written. Setting this to None means the function will return the graph as a string instead of saving it to a file.
    feature_names=X.columns, 
    class_names=["Not Churned", "Churned"], # "Not Churned" is for class 0, and "Churned" is for class 1. The intensity of the color indicates the proportion of samples in the node belonging to the predicted class.
    filled=True, # Colors the nodes based on the class they represent and the majority class in that node.
    # rounded=True, # Draws the nodes with rounded corners for better visual aesthetics.
    special_characters=True # Allows special characters (like ≥ or ≤) to be used in the text labels for nodes.
)

graph = Source(dot_data)
display(graph)

- Salary plays a significant role in predicting churn.
- Low salaries (≤ 64500) and high salaries (> 380000) lead to clearer splits.
- High entropy at certain nodes suggests these splits may not perfectly separate the classes.

# 6. Model Evaluation

- Accuracy: Overall prediction performance.
- Precision/Recall: For imbalanced datasets, recall is crucial to flag at-risk employees.
- Confusion Matrix: Identify false positives/negatives.

1. Given how important is salary, I would definitely love to have as a variable the salary the employee who quit was offered in the next job. Otherwise, things like: promotions or raises received during the employee tenure would be interesting.

2. The major findings are that employees quit at year anniversaries or at the beginning of the year. Both cases make sense. Even if you don’t like your current job, you often stay for 1 yr before quitting + you often get stocks after 1 yr so it makes sense to wait. Also, the beginning of the year is well known to be the best time to change job: companies are hiring more and you often want to stay until end of Dec to get the calendar year bonus.

3. Employees with low and high salaries are less likely to quit. Probably because employees with high salaries are happy there and employees with low salaries are not that marketable, so they have a hard time finding a new job.

In [ ]:
# df[df["churn"] == 1].sort_values("tenure_days")["tenure_days"].reset_index(drop=True).plot(kind="line")
# plt.title("Tenure Trend for Quit Employees")
# plt.xlabel("Index")
# plt.ylabel("Tenure (days)")
# plt.show()


In [ ]:
# df[df["churn"] == 1]["tenure_days"].reset_index(drop=True).plot(kind="line")

In [ ]:
# df.boxplot(column="tenure_days", vert=False, figsize=(10, 4), patch_artist=True, boxprops=dict(facecolor="skyblue"))
# plt.title("Boxplot of Tenure Days", fontsize=16)
# plt.xlabel("Tenure (days)", fontsize=12)
# plt.show()


In [ ]:
# df[df["churn"] == 1].groupby("tenure_days")["tenure_days"].count().plot(kind="line")